In [1]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.3 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Tue May 16 17:37:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from datasets.load import load_from_disk
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
import torch.optim as optim
from tqdm.notebook import tqdm
import torch.nn.functional as F
import csv
from transformers import TextDataset

# Set the device
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# For RTE

org_rte = load_from_disk('/content/drive/MyDrive/CS685/project/datasets/RTE')
rte = org_rte.copy()

In [6]:
rte

{'train': Dataset({
     features: ['sentence1', 'sentence2', 'label', 'idx'],
     num_rows: 21043
 }),
 'validation': Dataset({
     features: ['sentence1', 'sentence2', 'label', 'idx'],
     num_rows: 277
 }),
 'test': Dataset({
     features: ['sentence1', 'sentence2', 'label', 'idx'],
     num_rows: 3000
 })}

In [7]:
# rte["train"] = rte["train"][:1000]
# rte["validation"] = rte["validation"][:1000]

In [8]:
batch_size = 16

In [9]:
train_data = rte["train"]
validation_data = rte["validation"]

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)

from transformers import AutoTokenizer, AutoModelForSequenceClassification

bert_rte = AutoModelForSequenceClassification.from_pretrained("gchhablani/bert-base-cased-finetuned-rte").to(device)
teacher_tokenizer = AutoTokenizer.from_pretrained("gchhablani/bert-base-cased-finetuned-rte")

In [10]:
from torch.utils.data import DataLoader

# Create a DataLoader to generate batches
train_dataset = list(zip(rte['train']['sentence1'], rte['train']['sentence2'], rte['train']['idx']))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)

# Create a list to store the logits
logit_items = []

# Loop over the batches
for batch in tqdm(train_dataloader):
    # Unpack the batch
    sentence1, sentence2, idx = batch
    
    # Tokenize input sentences
    encoded_batch = teacher_tokenizer(sentence1, sentence2, padding = True, truncation = True, return_tensors = "pt").to(device)
    
    # Do a forward pass through the model to get logits
    outputs = bert_rte(**encoded_batch)
    
    # Extract the logits from the output and append to the list
    logits = outputs.logits.detach().cpu().numpy()
    for l in logits:
      logit_items.append(l)


  0%|          | 0/1316 [00:00<?, ?it/s]

In [11]:
logit_items

[array([-0.24485923,  0.0881362 ], dtype=float32),
 array([ 1.0182794, -0.9438952], dtype=float32),
 array([ 1.1458845, -1.0477064], dtype=float32),
 array([ 0.8116037, -0.5569402], dtype=float32),
 array([-0.01218496,  0.08513887], dtype=float32),
 array([ 1.1451035, -0.9881361], dtype=float32),
 array([ 1.0274947 , -0.94524455], dtype=float32),
 array([-1.7366784,  1.1843206], dtype=float32),
 array([ 0.45445922, -0.5084265 ], dtype=float32),
 array([ 0.821866 , -0.7455692], dtype=float32),
 array([ 0.84835196, -0.532085  ], dtype=float32),
 array([ 1.0465552, -0.9868009], dtype=float32),
 array([ 0.9292031, -0.8521389], dtype=float32),
 array([-0.85638016,  0.8441828 ], dtype=float32),
 array([-0.86955076,  0.49989796], dtype=float32),
 array([ 1.1343821, -0.85752  ], dtype=float32),
 array([-1.8582562,  1.3790468], dtype=float32),
 array([-0.867952 ,  0.8104612], dtype=float32),
 array([ 0.9390582, -0.9151605], dtype=float32),
 array([-1.9791805,  1.5236067], dtype=float32),
 array

In [12]:
from datasets import Dataset
import tensorflow as tf

def merge_true_and_teacher_logits(one_hot_labels, teacher_logits):
    return list(zip(one_hot_labels, teacher_logits))
    
new_dataset_dict = {"idx": rte['train']["idx"],
                    "sentence1": rte['train']["sentence1"],
                    "sentence2": rte['train']["sentence2"],
                    "label": rte['train']["label"],
                    "logits": logit_items,
                    "combined_logits": merge_true_and_teacher_logits(tf.one_hot(rte['train']["label"], depth=2), logit_items)}

new_dataset = Dataset.from_dict(new_dataset_dict)
new_dataset.save_to_disk("/content/drive/MyDrive/CS685/project/datasets/RTE/modified_train")

Saving the dataset (0/1 shards):   0%|          | 0/21043 [00:00<?, ? examples/s]

In [13]:
print(new_dataset['sentence1'][:4])
print(new_dataset['sentence2'][:4])
print(new_dataset['label'][:20])
a = []
for k in new_dataset['logits'][:20]:
  if k[0]>k[1]:
    a.append(0)
  else:
    a.append(1)

print(a)

['No Weapons of Mass Destruction Found in Iraq Yet.', 'A place of sorrow, after Pope John Paul II died, became a place of celebration, as Roman Catholic faithful gathered in downtown Chicago to mark the installation of new Pope Benedict XVI.', 'Herceptin was already approved to treat the sickest breast cancer patients, and the company said, Monday, it will discuss with federal regulators the possibility of prescribing the drug for more breast cancer patients.', 'Judie Vivian, chief executive at ProMedica, a medical service company that helps sustain the 2-year-old Vietnam Heart Institute in Ho Chi Minh City (formerly Saigon), said that so far about 1,500 children have received treatment.']
['Weapons of Mass Destruction Found in Iraq.', 'Pope Benedict XVI is the new leader of the Roman Catholic Church.', 'Herceptin can be used to treat breast cancer.', 'The previous name of Ho Chi Minh City was Saigon.']
[1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1]
[1, 0, 0, 0, 1, 0, 0, 

In [14]:
new_dataset

Dataset({
    features: ['idx', 'sentence1', 'sentence2', 'label', 'logits', 'combined_logits'],
    num_rows: 21043
})